https://colab.research.google.com/github/rufous86/studies/blob/main/ods_checks_structurization/extract_entities.ipynb

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!cp /content/drive/MyDrive/Study/courses/ods_checks_data/train_supervised_dataset.csv .

Mounted at /content/drive


In [2]:
!pip install natasha -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 79.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
import pandas as pd

df_train = pd.read_csv('train_supervised_dataset.csv')
df_train

,id,name,good,brand
0,0,Petmax Бантик леопард с красн розой 2шт,бантик,petmax
1,1,87191 Бусы для елки шарики_87191,бусы,NaN
2,2,Футболка Piazza Italia WR011446881,футболка,piazza italia
3,3,7) YI572-03X-ONE ЗАКОЛКА ДЛЯ ВОЛОС ДЛЯ ДЕВОЧКИ,заколка,NaN
4,4,Одежда (вес) 1500,одежда,NaN
...,...,...,...,...
24995,24995,"Вода ""Саирме"" с/г 500мл",вода,sairme
24996,24996,Моя Семя . 0.175л и ассортим,NaN,моя семья
24997,24997,Рулет бисквитн.Яшкино клубничный со слив,рулет,яшкино
24998,24998,460075794371 Почвогрунт Цветочное счастье Фас...,почвогрунт,фаско


In [7]:
text_list = [
    'Petmax Бантик леопард с красн розой 2шт',
    '87191 Бусы для елки шарики_87191',
    'Футболка Piazza Italia WR011446881',
    'YI572-03X-ONE ЗАКОЛКА ДЛЯ ВОЛОС ДЛЯ ДЕВОЧКИ',
    'Вода "Саирме" с/г 500мл',
             ]

In [8]:
from pprint import pprint

import natasha as ntsh
from natasha import (
    Segmenter,
    NewsEmbedding,
    NewsNERTagger,
    MorphVocab,
    NewsMorphTagger,
    Doc
)

segmenter = Segmenter()
emb = NewsEmbedding()
ner_tagger = NewsNERTagger(emb)
morph_tagger = NewsMorphTagger(emb)
morph_vocab = MorphVocab()

def extract_brands(text_list: list):
    """
    Функция для извлечения названий брендов из текста
    """
    res = {}
    for sent in text_list:
        res[sent] = []
        doc = Doc(sent)
        doc.segment(segmenter)
        doc.tag_ner(ner_tagger)
        for span in doc.spans:
            # print(span, span.type)
            if span.type == 'ORG':
                res[sent].append(span.text)
    return res

def extract_nouns(text_list: list):
    """
    Функция для извлечения только существительных из текста
    """
    res = {}
    for sent in text_list:
        sent = sent.lower()
        res[sent] = []
        doc = Doc(sent)
        doc.segment(segmenter)
        doc.tag_morph(morph_tagger)
        for token in doc.tokens:
            token.lemmatize(morph_vocab)
            if token.pos == 'NOUN':
                res[sent].append(token.lemma)
    return res

pprint(extract_brands(text_list))
pprint(extract_nouns(text_list))

{'87191 Бусы для елки шарики_87191': [],
 'Petmax Бантик леопард с красн розой 2шт': ['Petmax'],
 'YI572-03X-ONE ЗАКОЛКА ДЛЯ ВОЛОС ДЛЯ ДЕВОЧКИ': ['ДЕВОЧКИ'],
 'Вода "Саирме" с/г 500мл': [],
 'Футболка Piazza Italia WR011446881': []}
{'87191 бусы для елки шарики_87191': ['бусы', 'елка', 'шарики_87191'],
 'petmax бантик леопард с красн розой 2шт': ['леопард', 'роза', 'шт'],
 'yi572-03x-one заколка для волос для девочки': ['заколка', 'волос', 'девочка'],
 'вода "саирме" с/г 500мл': ['вода', 'саирма', 'г', 'мл'],
 'футболка piazza italia wr011446881': ['футболка']}
